In [8]:
import pydot
import pyparsing
import numpy as np
import csv
import os





def worstCheckEver(someString):
    if(someString[len(someString)-4]=="e"):#dotted
        return -18
    elif(someString[len(someString)-4]=="l"):#bold
        return -16
    else:
        return -2



def getEdgeList(edgeList):
    fullEdgeList=list()
    for j in range(len(edgeList)):
        edge_list=list()
        
        for i in range(len(edgeList[j])):
            pointList=list()
            edgeIndex=edgeList[j][i].to_string().find("->")
            
            point1=edgeList[j][i].to_string()[1:edgeIndex-1]
            hashtagIndex1=point1.find("#")
            point1=point1[hashtagIndex1+1:-1]

            
            s=worstCheckEver(edgeList[j][i].to_string())
            point2=edgeList[j][i].to_string()[edgeIndex+1:s]
            hashtagIndex2=point2.find("#")
            point2=point2[hashtagIndex2+1:]


            pointList.append(point1)
            pointList.append(point2)

            edge_list.append(pointList)

        fullEdgeList.append(edge_list)
    
    return fullEdgeList

    


    
    
def find_dstar(matrixpath):
    f=open(matrixpath,"r")

    array=[]
    for line in f:
        r=line.replace(" ", "")
        if(r[-1]=="1"):
            r=r[:-1]+"+"
        else:
            r=r[:-1]+"-"
        array.append(r.strip())


    row_length=len(array)
    column_length=len(array[0])


    Ncf=0
    Nuf=0
    Ncs=0
    dstarArray=[]
    for i in range(0,column_length):
        for j in range(0,row_length):
            isPassed=array[j][column_length-1]
            if isPassed == '-' and array[j][i]=='1':
                Ncf=Ncf+1

            elif isPassed=='-' and array[j][i]=='0':
                Nuf=Nuf+1
            elif isPassed=='+' and array[j][i]=='1':
                Ncs=Ncf+1
        if (Nuf+Ncs==0):
            dstarArray.append(-1)
        else:    
            dstar=(Ncf/(Nuf+Ncs))    
            dstarArray.append(dstar)
    return dstarArray


def findRanking(array):
    _,id = np.unique(np.asarray(array),return_inverse=True)
    result_array = (id.max() - id + 1).reshape(np.asarray(array).shape)
    return result_array





def get_suspiciousness_and_rank(gzoltarpath,projectName,methodNumberList,lineList):
    matchList=[]
    
    for i in range(len(methodNumberList)):
        matrixPath=""
        matrixPath=gzoltarpath+"/"+projectName+"/"+str(i+1)+"/matrix"
        suspiciousnessArray=find_dstar(matrixPath)#change according to your Suspiciousness finding method
        rankingArray=findRanking(suspiciousnessArray)
        yetAnotherList=[]
        
        
        maxSusp=-1
        if(methodNumberList[i]!=[]):
            
            for j in range(len(methodNumberList[i])):

                methodNumber=methodNumberList[i][j]
                if(suspiciousnessArray[methodNumber]>maxSusp):
                    d={}
                    d['methodNumber']=methodNumber
                    d['suspiciousness']=suspiciousnessArray[methodNumber]
                    d['ranking']=rankingArray[methodNumber]
                    d['percentage']=(rankingArray[methodNumber])/lineList[j]

        
            matchList.append(d)
        else:
            d={}
            d['methodNumber']=""
            d['suspiciousness']=""
            d['ranking']=""
            d['percentage']=""
            matchList.append(d)
    return matchList        
    




def findBugs(subNode,color):
    lineNumberList=list()
    for node in subNode:
        
        nodeColor=node.get_fillcolor()
        if(nodeColor.find(color)!=-1):
            nodeName=node.get_name()
            hashtagIndex=nodeName.find("#")
            if(hashtagIndex!=-1):
                lineNumber=nodeName[hashtagIndex+1:-1]
                lineNumberList.append(lineNumber)
    
    return lineNumberList

def getNodeAndEdgeList(projectName,path,versionCount):
    edgeList=list()
    nodeList=list()
    
    
    for i in range(0,versionCount):
        projectPath=""
        projectPath=path+projectName+"/"+str(i+1)+".dot"
        graph = pydot.graph_from_dot_file(projectPath)
        edgeList.append(graph[0].get_edge_list())

        nodeList.append(graph[0].get_nodes())
    return nodeList,edgeList

def findBuggyLines(nodeList):
    bugList=list()
    for subNode in nodeList:
        lineNumberList=findBugs(subNode,"red")
        if lineNumberList:

            bugList.append(lineNumberList)        
        else:
            bugList.append([])
      #      lineNumberList=findBugs(subNode,"orange")
      #      if lineNumberList:
      #          bugList.append(lineNumberList)        
      #      else:
      #          lineNumberList=findBugs(subNode,"yellow")
      #          bugList.append(lineNumberList)
    return bugList        



   

def findMethodNumber(gzoltarpath,projectName,bugList):
    methodNumberList=list()
    lineList=list()
    for i in range(len(bugList)):
        spectraPath=""
        spectraPath=gzoltarpath+"/"+projectName+"/"+str(i+1)+"/spectra"
        lineNumberList=list()
        k=1
        with open(spectraPath, "r") as ins:
            
            for line in ins:
                
                hashtagIndex=line.find("#")
                if(hashtagIndex!=-1):
                    lineNumber=line[hashtagIndex+1:-1]
                    for j in range(len(bugList[i])):
                        if(bugList[i][j]==lineNumber):
                            lineNumberList.append(k)
                k=k+1
        lineList.append(k-1)
        methodNumberList.append(lineNumberList)
    return(methodNumberList,lineList)    


#making graph list
path="/home/fmert/Praktikum/graphs/"
gzoltarpath="/home/fmert/fault-localization-data/gzoltar/gzoltars"
projectName="Lang"
versionCount=65
nodeList,edgeList=getNodeAndEdgeList(projectName,path,versionCount)
bugList=findBuggyLines(nodeList)

methodNumberList,lineList=findMethodNumber(gzoltarpath,projectName,bugList)

matchList=get_suspiciousness_and_rank(gzoltarpath,projectName,methodNumberList,lineList)
edgeList=getEdgeList(edgeList)

currentPath=os.getcwd()
csv_file=currentPath+"/deneme2.csv"

csv_columns=['methodNumber','suspiciousness','ranking','percentage']

matchList_writer(csv_file,csv_columns,matchList)


print(nodeList[0][0].get_name())


"ochiai"
